## Testing notebook playground for extracting data

In [199]:
import pandas as pd
import mysql.connector as msc
import json
import tweepy
import csv
import re
import string
import requests
import pprint
import datetime
from bs4 import BeautifulSoup

In [3]:
# read config file into JSON object
configFile = open('config.json')
config = json.load(configFile)

# make connection to the GCloud database
db_config = config['database']
connection = msc.connect(
    host=db_config['host'], 
    port=db_config['port'],
    user=db_config['user'],
    password=db_config['password'],
    database=db_config['database']
)

# authorize Twitter app with Tweepy OAuthHandler
twit_config = config['twitter']
auth = tweepy.OAuthHandler(twit_config['api_key'], twit_config['api_key_secret'])
auth.set_access_token(twit_config['access_token'], twit_config['access_token_secret'])
twitapi = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
# extract Tweets from Malaysian MOH about Malaysian COVID-19 case data
# write to pandas database
tweet_collection = twitapi.search("KKMPutrajaya")

15


In [82]:
# Get user id of @KKMPutrajaya
url = "https://api.twitter.com"
headers = {"Authorization": "Bearer {}".format(twit_config['bearer_token'])}
response = requests.request("GET", url + "/2/users/by/username/KKMPutrajaya", headers=headers)
user_id = response.json()['data']['id']

In [44]:
# initialize pandas data set
tweets = []
next_tokens = []

In [81]:
# get posts from Malaysian MOH (@KKMPutrajaya)
search_url = (url + "/2/users/{}/tweets".format(user_id))
response = requests.request("GET", search_url, headers=headers)
data = response.json()['data']

In [47]:
# fetch tweet data
# this fetches thousands of MOH's tweets, which are factored into the quota for my 
# Twitter account, so beware!
def fetch_data():
    # get posts from Malaysian MOH (@KKMPutrajaya)
    search_url = (url + "/2/users/{}/tweets".format(user_id))
    response = requests.request("GET", search_url, headers=headers)
    data = response.json()['data']

    # cycle and add to data
    for tweet in data:
        if "Status Terkini #COVID19" in tweet['text']:
            tweets.append(tweet)
    # get next pagination token
    next_token = response.json()['meta'].get('next_token')
    next_tokens.append(next_token)
    # while next token is not None (ie there is still a next page)
    while next_token is not None:
        # get next page
        search_url = (url + "/2/users/{}/tweets?pagination_token={}".format(user_id, next_token))
        response = requests.request("GET", search_url, headers=headers)
        data = response.json()['data']
        next_token = response.json()['meta'].get('next_token')
        # cycle through the next page
        for tweet in data:
            if "Status Terkini #COVID19" in tweet['text']:
                tweets.append(tweet)
                next_tokens.append(next_token)

In [80]:
tweetsdf = pd.DataFrame(tweets)

In [83]:
next_tokensdf = pd.DataFrame(next_tokens)

In [79]:
search_url = (url + "/2/users/{}/tweets?end_time={}".format(user_id, "2021-03-11T12:00:00Z"))
response = requests.request("GET", search_url, headers=headers)

In [77]:
# function to cycle through pages automatically
print(next_token)
def goForward():
    global next_token
    if next_token is None:
        raise Exception("no next page")
    else:
        search_url = (url + "/2/users/{}/tweets?end_time={}&pagination_token={}".format(user_id, "2021-03-11T12:00:00Z", next_token))
        response = requests.request("GET", search_url, headers=headers)
        next_token = response.json()['meta'].get('next_token')
        print(response.json()['data'])

None


In [98]:
example_tweet = tweetsdf.iloc[0, :]['text']

In [110]:
# parser for tweet content
def parseTweetContent(tweet):
    # get tweet content without \n and 
    # explode it via spaces
    exploded_tweet = re.split('\n| ', tweet)
    
    # get date as "DD MMM YYYY", where the month is in Malay
    date = "{} {} {}".format(exploded_tweet[3], exploded_tweet[4], exploded_tweet[5])

    

In [108]:
# example tweet content
parseTweetContent(example_tweet)

['Status',
 'Terkini',
 '#COVID19,',
 '21',
 'Jun',
 '2021',
 '',
 'Kes',
 'sembuh=',
 '5,439',
 'Jumlah',
 'kes',
 'sembuh=',
 '633,624',
 'Kes',
 'baharu',
 'positif=',
 '4,611',
 '(8',
 'import)',
 '',
 'Jumlah',
 'positif=',
 '701,019',
 'Kes',
 'kematian=',
 '69',
 'Jumlah',
 'kes',
 'kematian=',
 '4,477',
 'Kes',
 'dirawat',
 'di',
 'ICU=',
 '880',
 'Bantuan',
 'alat',
 'pernafasan=',
 '452',
 'https://t.co/zUQch1MbEs']

### Testing out web scraper for kpkesihatan.com instead to fetch data

In [200]:
# function to convert python date to DD-MMM-YYYY, where "MMM" is the month in malay
month_dict = [
    'januari',
    'februari',
    'mac',
    'april',
    'mei',
    'jun',
    'julai',
    'ogos',
    'september',
    'oktober',
    'november',
    'disember'
]
def convertDate(date):
    return date.strftime("%d-{}-%Y".format(month_dict[int(date.strftime("%m"))-1]))

In [204]:
[convertDate(datetime.datetime(2021, i, 21)) for i in range(1, 13)]

['21-januari-2021',
 '21-februari-2021',
 '21-mac-2021',
 '21-april-2021',
 '21-mei-2021',
 '21-jun-2021',
 '21-julai-2021',
 '21-ogos-2021',
 '21-september-2021',
 '21-oktober-2021',
 '21-november-2021',
 '21-disember-2021']

In [207]:
# can do formatting on this later
def makeURL(date):
    return "https://kpkesihatan.com/{}/kenyataan-akhbar-kpk-{}-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia/".format(
        date.strftime("%d/%m/%Y"), 
        convertDate(date)
    )

In [208]:
makeURL(datetime.datetime(2021, 6, 21))

'https://kpkesihatan.com/21/06/2021/kenyataan-akhbar-kpk-21-jun-2021-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia/'

In [119]:
# get content of website
response = requests.get(myurl)
# initialize printer
pp = pprint.PrettyPrinter(indent=4)

In [146]:
# create soup object
soup = BeautifulSoup(response.content, 'html.parser')
# get all bulleted lists
result = soup.find_all('ul')
# the case data will be the second element
result[1]

<ul><li>Kes sembuh : <strong>5,941 kes </strong>(628,185 kes kumulatif, 90.2%);</li><li>Kes baharu :<strong> 5,293 kes</strong> (696,408 kes kumulatif);</li><li>Kes import : <strong>7 kes </strong>(3 warganegara, 4 bukan warganegara);</li><li>Kes tempatan : <strong>5,286 kes </strong>[3,894 warganegara (73.7%); 1,392 bukan warganegara (26.3%)];</li><li>Kes aktif :<strong> 63,815 kes</strong>;</li><li>Kes yang memerlukan rawatan di Unit Rawatan Rapi (ICU) : <strong>880 kes</strong>;</li><li>Kes memerlukan bantuan pernafasan : <strong>454 kes</strong>;</li><li>Kes kematian :<strong> 60 kes </strong>(4,408 kes kumulatif, 0.63%; 56 warganegara, 4 bukan warganegara);</li><li>Jumlah kluster : <strong>2,623 kluster</strong>;</li><li>Jumlah kluster baharu :<strong> 19 kluster</strong>;</li><li>Jumlah kluster yang telah tamat :<strong>1,789 kluster</strong>; dan</li><li>Jumlah kluster aktif : <strong>834 kluster</strong>.</li></ul>

In [173]:
# Create soup specifically for the response content
soup_cases = BeautifulSoup(str(result[1]), 'html.parser')
# Create variable to (Malay) text mapping
vtt_map = {
    'cured_cases': 'Kes sembuh',
    'new_cases': 'Kes baharu',
    'import_cases': 'Kes import',
    'local_cases': 'Kes tempatan',
    'active_cases': 'Kes aktif',
    'resp_asst_cases': 'Kes yang memerlukan rawatan',
    'death_cases': 'Kes kematian',
    'number_of_clusters': 'Jumlah kluster',
    'number_of_new_clusters': 'Jumlah kluster baharu',
    'number_of_expired_clusters': 'Jumlah kluster yang telah tamat',
    'number_of_active_clusters': 'Jumlah kluster aktif'
}

In [192]:
# Fetch data into dict object
day_data = {}
for var in vtt_map:
    # Get the string containing the key words
    keyword_string = str(soup_cases.find(lambda tag: tag.name == "li" and vtt_map[var] in tag.text))
    # Get the soup for the specific string
    soup_var = BeautifulSoup(keyword_string, 'html.parser')
    # use the custom soup to extract the case data, which is stored into the day_data object
    content = soup_var.find(lambda tag: tag.name == "strong").contents[0]
    # parse the content to only extract the integer value for the corresponding variable
    for s in ["\xa0", "kes", "kluster", " ", ","]:
        content = content.replace(s, "")
    # convert value to int
    day_data[var] = int(content)

In [193]:
day_data

{'cured_cases': 5941,
 'new_cases': 5293,
 'import_cases': 7,
 'local_cases': 5286,
 'active_cases': 63815,
 'resp_asst_cases': 880,
 'death_cases': 60,
 'number_of_clusters': 2623,
 'number_of_new_clusters': 19,
 'number_of_expired_clusters': 1789,
 'number_of_active_clusters': 834}